# Comic Book Generation

Diffusion models, especially in combination with text-to-image conditioning, are excellent for creating diverse environments and backgrounds that open up exciting new possibilities for artists and creators. In this project, you will be creating comic book pages with AI using techniques like DreamBooth, LoRA to bring the comic world to life. 
These advanced methods allow you to generate high-quality, consistent characters, scenes, and stylistic elements, all with minimal input. Here's a brief overview of how these techniques can be used in comic book generation:

- **DreamBooth** (https://arxiv.org/abs/2208.12242) for Character Consistency: In comic creation, keeping characters consistent across different scenes and expressions is key. DreamBooth allows you to add specific characters to a pre-trained model, fine-tuning it with just a few images of each character. This way, you can introduce new, unique characters into the model’s vocabulary and generate multiple scenes featuring them with consistent features, expressions, and poses.

- **Low-Rank Adaptation (LoRA)** (https://arxiv.org/abs/2106.09685) for Style and Atmosphere: Using LoRA, you can fine-tune models to adapt specific stylistic elements (like a unique art style or theme) without retraining the entire model. LoRA allows you to apply fine-tuned style elements to multiple comic panels, creating a cohesive and immersive visual experience that aligns with the specific mood or genre of your story.

By conditioning the final trained model on specific prompts, you can generate detailed backgrounds—such as urban landscapes, mystical forests, or futuristic spaces—allowing each panel to visually support the story while maintaining stylistic unity.

In [ ]:
#Import required libraries
# NOTE: include the annotations import if you are facing some typing issues
# from __future__ import __annotations__

import itertools
import math
import os
import glob
from pathlib import Path

import torch
import torch.utils.checkpoint
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from diffusers import AutoencoderKL, DDPMScheduler, DDIMScheduler, StableDiffusionPipeline, UNet2DConditionModel
from PIL import Image
from torchvision import transforms
from tqdm.autonotebook import tqdm
from transformers import CLIPTextModel
from transformers import AutoTokenizer
from transformers import pipeline, Pipeline
from peft import LoraConfig, get_peft_model, PeftModel

# suppress warnings
import warnings
warnings.filterwarnings("ignore")

# suppress logs from transformers library
import logging
logging.getLogger("transformers.tokenization_utils_base").setLevel(logging.ERROR)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if str(device) != "cuda":
    print("No CUDA found. Please check your dependencies")

## Image To Be Trained

We use only a small number of images to fine tune a stable diffusion model.

In [ ]:
def image_grid(imgs: list[Image.Image], rows: int, cols: int, resize: int = 256) -> Image.Image:
    assert len(imgs) == rows * cols

    if resize is not None:
        imgs = [img.resize((resize, resize)) for img in imgs]
    w, h = imgs[0].size
    grid = Image.new("RGB", size=(cols * w, rows * h))

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i % cols * w, i // cols * h))
    return grid


image_dir = "./images"
imgs = [Image.open(path) for path in glob.glob(os.path.join(image_dir, "*.jpg"))] #change the path to the folder where the images are stored
image_grid(imgs[:6], 1, 6)

### Image captioning

In [ ]:
# [5 points] TODO: Implement an image captioning method.
pipe = pipeline("image-to-text", model="Salesforce/blip-image-captioning-large", device=device)
def caption_image(image) -> str:
    ...

### Prepare Diffusion Models

In [3]:
pretrained_model_name_or_path = "stabilityai/stable-diffusion-2-1-base"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path,
    subfolder="tokenizer",
    use_fast=False,
)
text_encoder = CLIPTextModel.from_pretrained(
    pretrained_model_name_or_path, subfolder="text_encoder"
)
vae = AutoencoderKL.from_pretrained(
    pretrained_model_name_or_path, subfolder="vae"
)
unet = UNet2DConditionModel.from_pretrained(
    pretrained_model_name_or_path, subfolder="unet"
)

# NOTE: Try different schedulers for fine-tunning.
noise_scheduler = DDPMScheduler(
    beta_start=0.00085,
    beta_end=0.012,
    beta_schedule="scaled_linear",
    num_train_timesteps=1000,
)


### Tokenizer

In [4]:
# [5 points] TODO: Implement the function below to obtain the input id token tensor from a string, and vice versa.
def tokenize_prompt(tokenizer, prompt: str) -> torch.Tensor:
    ...


def decode_tokens(tokens: torch.Tensor) -> str:
    ...

In [ ]:
# Test cases
long_prompt_str = (
    "This is the first image description. A superhero flying in the sky."
    "This is the first image description. A superhero flying in the sky."
    "This is the first image description. A superhero flying in the sky."
    "This is the first image description. A superhero flying in the sky."
    "This is the first image description. A superhero flying in the sky."
    "The fourth image is the resolution of the story."
)

tokens = tokenize_prompt(tokenizer, long_prompt_str)
output_str = decode_tokens(tokens[0])
print(tokens)
print(output_str)

Question:

- [5 points] Try encode `prompt_str = "happy"`, `prompt_str = "happy girl"` and `prompt_str = "happy gi555rl"`, what is the shape of the output tensor, and what tokens you got?

- [5 points] What if having a long text such as the provided `long_prompt_str`? Will the tokenizer capture all the content?


### Dreambooth Dataset

We build a dataset to prepare the instance and class images with the prompts for fine-tuning the model.

In [6]:
# [5 points] TODO: implement the missing functions
class DreamBoothDataset(Dataset):
    """
    A dataset to prepare the images with the prompts for fine-tuning the model.
    It pre-processes the images and the tokenizes prompts.
    """

    def __init__(
        self,
        instance_data_root,
        tokenizer,
        size: int = 512,
        instance_prompt: str | None = None,
        instance_prompt_model: Pipeline | None = None,
        instance_prompt_model_token: str | None = None,
    ):
        self.size = size
        self.tokenizer = tokenizer

        self.instance_data_root = Path(instance_data_root)
        if not self.instance_data_root.exists():
            raise ValueError("Instance images root doesn't exists.")

        self.instance_images_path = sorted(list(Path(instance_data_root).iterdir()))
        self.num_instance_images = len(self.instance_images_path)
        if instance_prompt_model is not None:
            self.instance_prompt = self.obtain_caption_for_each_image(
                instance_prompt_model, self.instance_images_path, instance_prompt_model_token)
        else:
            assert instance_prompt is not None
            self.instance_prompt = instance_prompt
        self._length = self.num_instance_images

        self.image_transforms = transforms.Compose(
            [
                transforms.Resize(size, interpolation=transforms.InterpolationMode.BILINEAR),
                transforms.ToTensor(),
                transforms.Normalize([0.5], [0.5]),
            ]
        )

    # TODO: obtain image captions for each image
    def obtain_caption_for_each_image(self, pipeline, image_paths, token):
        ...

    def __len__(self):
        return self._length

    def __getitem__(self, index):
        example = {}
        instance_image = Image.open(self.instance_images_path[index % self.num_instance_images])
        if not instance_image.mode == "RGB":
            instance_image = instance_image.convert("RGB")
        example["instance_images"] = self.image_transforms(instance_image)
        if isinstance(self.instance_prompt, str):
            instance_prompt = self.instance_prompt
        elif isinstance(self.instance_prompt, list) and isinstance(self.instance_prompt[0], str):
            instance_prompt = self.instance_prompt[index]
        else:
            raise ValueError

        # TODO: tokenize prompt
        example["instance_prompt_ids"] = ...
        example["instance_prompt"] = instance_prompt

        return example


def collate_fn(examples):
    input_ids = [example["instance_prompt_ids"] for example in examples]
    pixel_values = [example["instance_images"] for example in examples]

    pixel_values = torch.stack(pixel_values)
    pixel_values = pixel_values.to(memory_format=torch.contiguous_format).float()

    input_ids = torch.cat(input_ids, dim=0)

    batch = {
        "input_ids": input_ids,
        "pixel_values": pixel_values,
    }
    return batch


In [ ]:
# Test cases
dataset = DreamBoothDataset(
    instance_data_root=image_dir,
    instance_prompt=f"a photo of <AMA> girl",
    tokenizer=tokenizer,
)
dataitem = dataset[0]
print("Test case one:")
print(dataitem["instance_images"].shape, dataitem["instance_prompt_ids"].shape, dataitem["instance_prompt"])

pipe = pipeline("image-to-text", model="Salesforce/blip-image-captioning-large", device=device)
dataset = DreamBoothDataset(
    instance_data_root=image_dir,
    instance_prompt=f"a photo of <AMA> girl",
    tokenizer=tokenizer,
    instance_prompt_model=pipe,
    instance_prompt_model_token="<AMA>",
)

dataitem = dataset[0]
print("Test case two:")
print("All prompts", dataset.instance_prompt)
print(dataitem["instance_images"].shape, dataitem["instance_prompt_ids"].shape, dataitem["instance_prompt"])

In [8]:
# Housekeeping
import gc
del pipe
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

### LoRA


- LoRA layers are added to the UNet's attention layers (QKV and output projection).
- Only the additional LoRA layers are trained during fine-tuning.


In [ ]:
# [5 points] TODO: obtain image captions for each image
def lora_setup(unet, text_encoder, train_text_encoder: bool = False):
    # TODO: define LoRA parameters
    lora_rank_unet = ...
    lora_alpha_unet = ...
    lora_rank_text_encoder = ...
    lora_alpha_text_encoder = ...

    config = LoraConfig(
        r=lora_rank_unet,
        lora_alpha=lora_alpha_unet,
        target_modules=["to_q", "to_v"],
    )
    unet = get_peft_model(unet, config)
    unet.print_trainable_parameters()

    # TODO: setup LoRA for the text encoder.
    # You may setup your own target modules by exploring from the text encoder architecture.
    if train_text_encoder:
        config = ...
        text_encoder = get_peft_model(text_encoder, config)
        text_encoder.print_trainable_parameters()

    return unet, text_encoder


unet, text_encoder = lora_setup(unet, text_encoder, train_text_encoder=True)

Question:

- [5 points] Observe what happens before and after setting up LoRA. What modules are added? What are the shapes of the added modules?
- [5 points] Describe the computation mechanism of LoRA.

### DreamBooth:

- A custom dataset is created with the instance images and prompts.
- The model is fine-tuned by adding noise to the latents, followed by a forward pass through the UNet, using the instance prompts as conditioning.

In [10]:
# [10 points] TODO: Fill in the missing lines in the training function
def train(text_encoder, vae, unet, hyperparameters, train_dataset):
    torch.manual_seed(hyperparameters["seed"])

    train_batch_size = hyperparameters["train_batch_size"]
    gradient_accumulation_steps = hyperparameters["gradient_accumulation_steps"]
    learning_rate = hyperparameters["learning_rate"]
    max_train_steps = hyperparameters["max_train_steps"]
    output_dir = hyperparameters["output_dir"]
    gradient_checkpointing = hyperparameters["gradient_checkpointing"]
    train_text_encoder = hyperparameters["train_text_encoder"]

    unet, text_encoder = lora_setup(unet, text_encoder, train_text_encoder=train_text_encoder)

    if gradient_checkpointing:
        unet.enable_gradient_checkpointing()

    train_dataloader = DataLoader(
        train_dataset, batch_size=train_batch_size, shuffle=True, collate_fn=collate_fn)

    # Initialize the optimizer
    optimizer = torch.optim.AdamW(
        itertools.chain(unet.parameters(), text_encoder.parameters()) if train_text_encoder else unet.parameters(),
        lr=learning_rate,
    )

    # Keep vae in eval mode as we don't train it
    vae.eval()
    vae.requires_grad_(False)
    # Move vae and unet to device
    vae.to(device)
    unet.to(device)
    text_encoder.to(device)

    unet.train()
    if train_text_encoder:
        text_encoder.train()
    else:
        text_encoder.eval()

    # We need to recalculate our total training steps as the size of the training dataloader may have changed.
    num_update_steps_per_epoch = math.ceil(len(train_dataloader) / gradient_accumulation_steps)
    num_train_epochs = math.ceil(max_train_steps / num_update_steps_per_epoch)

    # Only show the progress bar once on each machine.
    progress_bar = tqdm(range(max_train_steps))
    progress_bar.set_description("Steps")
    global_step = 0

    accumulated_loss = 0
    loss_running = []
    for epoch in range(num_train_epochs):
        for i, batch in enumerate(train_dataloader):

            # Convert images to latent space
            batch["pixel_values"] = batch["pixel_values"].to(device, dtype=vae.dtype)
            latents = vae.encode(batch["pixel_values"]).latent_dist.sample()
            latents = latents * vae.config.scaling_factor

            # Sample noise that we'll add to the latents
            # TODO: sample a random noise. What should be the shape of the noise?
            noise = ...
            bsz = latents.shape[0]

            timesteps = torch.randint(0, noise_scheduler.num_train_timesteps, (bsz,), device=noise.device).long()  # TODO: Sample a random integer as the timestep for each image. 
            # Hint: use `torch.randint()`.
            # The shape of the tensor should be (bsz, ).

            # TODO: Add noise to the latents
            noisy_latents = noise_scheduler.add_noise(
                ...
            )

            # TODO: Enocde your tokens with text encoder
            encoder_hidden_states = ...

            # Predict the noise residual
            noise_pred = unet(noisy_latents, timesteps, encoder_hidden_states)

            # Get the target for loss depending on the prediction type
            if noise_scheduler.config.prediction_type == "epsilon":
                target = noise
            elif noise_scheduler.config.prediction_type == "v_prediction":
                target = noise_scheduler.get_velocity(latents, noise, timesteps)
            else:
                raise ValueError(f"Unknown prediction type {noise_scheduler.config.prediction_type}")

            # TODO: Calculate the MSE loss
            loss = ...

            accumulated_loss = loss + accumulated_loss
            if (i + 1) % gradient_accumulation_steps == 0:
                optimizer.zero_grad()
                accumulated_loss.backward()

                optimizer.step()
                accumulated_loss = 0

                progress_bar.update(1)
                global_step += 1

                if global_step >= max_train_steps:
                    break

            loss_running.append(loss.detach().item())
            logs = {"loss": sum(loss_running[-100:]) / min(len(loss_running), 100)}
            progress_bar.set_postfix(**logs)

    output_path = os.path.join(output_dir, "checkpoint-{}_{}".format(global_step, hyperparameters["learning_rate"]))

    unet.save_pretrained(
        os.path.join(output_path, "unet"), state_dict=unet.state_dict()
    )
    text_encoder.save_pretrained(
        os.path.join(output_path, "text_encoder"), state_dict=text_encoder.state_dict()
    )


In [ ]:
# Setting up all training hyperparameters.
# You can keep the other hyperparameters as they are, 
# but you can also try to change them to see if you can get better results.
placeholder_token = "<UnicornGirl>"

# TODO: you may use a captioning model
# train_pipe = pipeline("image-to-text", model="Salesforce/blip-image-captioning-large", device=device)
train_pipe = None
train_dataset = DreamBoothDataset(
    instance_data_root=image_dir,
    instance_prompt=f"a photo of {placeholder_token} girl",
    tokenizer=tokenizer,
    instance_prompt_model=train_pipe,
    instance_prompt_model_token=placeholder_token,
)

# TODO: you may tweak those parameters for fine-tuning.
hyperparameters = {
    "learning_rate": 1e-4,
    "max_train_steps": 1000,
    "train_batch_size": 1,
    "gradient_accumulation_steps": 2,
    "gradient_checkpointing": True,
    "seed": 20,
    "output_dir": f"./output/{placeholder_token}",
    "train_text_encoder": True,
}
train(text_encoder, vae, unet, hyperparameters, train_dataset)

Question:

- [10 points] What would happen if you change from DDPM scheduler to DDIM scheduler? and why?
- [10 points] What would happen if you use a smaller rank of LoRA? (e.g. 4)
- [10 points] Which prompt strategy works best? Is using enriched prompts by a caption model helpful? and why?

Please play with different settings and save your resulting images.

In [12]:
# Clear the memory here
import gc

del text_encoder, vae, unet
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

### Load your model and generate sample images

In [ ]:
steps = hyperparameters["max_train_steps"]
pipe_path = os.path.join(hyperparameters["output_dir"], "checkpoint-{}_{}".format(steps, hyperparameters["learning_rate"]))

pipe = StableDiffusionPipeline.from_pretrained(
    pretrained_model_name_or_path,
    torch_dtype=torch.float16,
)

unet_sub_dir = os.path.join(pipe_path, "unet")
text_encoder_sub_dir = os.path.join(pipe_path, "text_encoder")

pipe.unet = PeftModel.from_pretrained(pipe.unet, unet_sub_dir, adapter_name="my_custom_lora")
pipe.text_encoder = PeftModel.from_pretrained(pipe.text_encoder, text_encoder_sub_dir, adapter_name="my_custom_lora")

pipe.unet.half()
pipe.text_encoder.half()

pipe.to(device)
print(f"Load the pipeline from {pipe_path}.")


In [ ]:
# Run the cell and see if the generated images include the corgi in the reference images.
# If you want to train the model with new hyperparameters, 
# consider restart the runtime and run all cells again.

def generate(prompt, num_samples=4, num_rows=1, seed=None):
    torch.manual_seed(seed)
    new_images = [] 
    for _ in range(num_rows):
        with torch.no_grad():
            images = pipe([prompt] * num_samples, num_inference_steps=50, guidance_scale=7.5).images
        new_images.extend(images)
    return new_images


prompt = "one {}".format(placeholder_token)
new_images = generate(prompt, seed=888)
image_grid(new_images, 1, 4)

In [29]:
from PIL import Image, ImageDraw, ImageFont, ImageOps
import textwrap


# Function to create the comic page
def create_comic_page(images, descriptions, title, image_size=(300, 300), font_path=None):
    # Define layout parameters
    width, height = image_size
    border_size = 10  # Border around each image
    title_height = 100  # Title height
    gap_between_rows = 30  # Extra gap between rows
    desc_height = 50  # Height allocated for description under each image
    side_padding = 10  # White space reserved on both left and right sides
    column_padding = 20  # Padding between the first and second columns
    comic_width = 2 * (width + 2 * border_size) + column_padding + 2 * side_padding  # Adjust width with side and column padding
    comic_height = 2 * height + title_height + gap_between_rows + 2 * desc_height + 4 * border_size
    
    # Create a new blank image for the comic page with white background
    comic_page = Image.new('RGB', (comic_width, comic_height), 'white')
    draw = ImageDraw.Draw(comic_page)
    
    # Font for the title and descriptions
    if font_path:
        title_font = ImageFont.truetype(font_path, 60)  # Bigger font for title
        desc_font = ImageFont.truetype(font_path, 20)
    else:
        title_font = ImageFont.load_default(40)
        desc_font = ImageFont.load_default(15)
    
    # Draw the title at the top center of the comic page
    title_bbox = draw.textbbox((0, 0), title, font=title_font)
    title_width = title_bbox[2] - title_bbox[0]
    title_x_offset = (comic_width - title_width) // 2
    draw.text((title_x_offset, 20), title, fill='black', font=title_font)
    
    # Loop through images and descriptions
    for idx, (img, desc) in enumerate(zip(images, descriptions)):
        # Resize image to fit the grid and add border
        img = img.resize(image_size)
        img_with_border = ImageOps.expand(img, border=border_size, fill='black')
        
        # Calculate position in the grid (row and column)
        col = idx % 2
        row = idx // 2
        
        # Position of the image in the comic page
        x_offset = col * (width + 2 * border_size + column_padding // 2) + side_padding
        y_offset = row * (height + gap_between_rows + desc_height) + title_height
        
        # Paste the image with the border in the comic page
        comic_page.paste(img_with_border, (x_offset, y_offset))
        
        # Draw the description below the first two images directly
        desc_x_offset = x_offset + 10
        desc_y_offset = y_offset + height + border_size + 10
        
        desc_text = textwrap.fill(desc, width=47)  # Wrap text
        draw.text((desc_x_offset, desc_y_offset), desc_text, fill='black', font=desc_font)

    return comic_page

### Make up your stroy for a comic book page

In [ ]:
# [20 points] TODO: Create your own storyline and images.
title = "The Fallen UnicornGirl"
# NOTE: You may use `generate(prompt, seed=888)` in the previous cell to tweak for
# a proper prompt and seed to fill the story board.
story = [
    {
        "prompt": "one {} wakes up".format(placeholder_token),
        "seed": 2,
        "image_index": 3,
        "description": "The girl wakes up.",
    },
    {
        "prompt": "two {}, one <UnicornGirl> and one without unicorn".format(placeholder_token),
        "seed": 2,
        "image_index": 3,
        "description": "She is having fun with her friend.",
    },
    {
        "prompt": "one {} and a witch".format(placeholder_token),
        "seed": 888,
        "image_index": 0,
        "description": "Her friend turns to be red-eye witch in all of a sudden!!!!",
    },
    {
        "prompt": "one {} and a dragon".format(placeholder_token),
        "seed": 888,
        "image_index": 1,
        "description": "The girl has been cursed to be a ugly dragon",
    },
]
images = [generate(it["prompt"], seed=it["seed"])[it["image_index"]] for it in story]
descriptions = [it["description"] for it in story]

create_comic_page(images, descriptions, title)